In [1]:
import cv2
import depthai as dai
import numpy as np

import time
from pathlib import Path
from matplotlib import pyplot as plt


In [2]:
pipeline = dai.Pipeline()

cam_left = pipeline.createMonoCamera()
cam_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

xout_left = pipeline.createXLinkOut()
xout_left.setStreamName('left')
cam_left.out.link(xout_left.input)


In [4]:
image = cv2.imread('cv.png')

cv2.imshow('Image input', image)
cv2.waitKey(1000)
cv2.destroyAllWindows()            

bw_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
bw_image = np.float32(bw_image)

dest = cv2.cornerHarris(bw_image, 2, 5, 0.07)
 
dest = cv2.dilate(dest, None)
 
image[dest > 0.01 * dest.max()]=[0, 0, 255]
 
cv2.imshow('Image with Borders', image)
cv2.waitKey(1000)
cv2.imwrite("harris_corner_detection.png", image)
cv2.destroyAllWindows()


In [5]:
image = cv2.imread('cv.png')

cv2.imshow('Image original', image)
cv2.waitKey(1000)
cv2.destroyAllWindows()

edges_img = cv2.Canny(image,100,200)

cv2.imshow('Image with edges', edges_img)
cv2.waitKey(1000)
cv2.imwrite("canny_edge_detection.png", edges_img)
cv2.destroyAllWindows()


In [6]:
img1 = cv2.imread('sruthi_video/16696818566204.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

img2 = cv2.imread('sruthi_video/16696818566549.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)


In [7]:
sift = cv2.SIFT_create() 

kp_img1, desc_img1 = sift.detectAndCompute(img1, None) 
kp_img2, desc_img2 = sift.detectAndCompute(img2, None) 

bf = cv2.BFMatcher()
matches = bf.knnMatch(desc_img1, desc_img2, k=2)

good_points=[]     
for m, n in matches: 
    if(m.distance < 0.6*n.distance): 
        good_points.append(m) 


In [8]:
query_pts = np.float32([kp_img1[m.queryIdx] 
                .pt for m in good_points]).reshape(-1, 1, 2) 
 
train_pts = np.float32([kp_img2[m.trainIdx] 
                .pt for m in good_points]).reshape(-1, 1, 2) 
 
matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 
 
matches_mask = mask.ravel().tolist() 

h,w = img1.shape
 
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
 
dst = cv2.perspectiveTransform(pts, matrix)
 
homography = cv2.polylines(img2, [np.int32(dst)], True, (255, 0, 0), 3) 
 
cv2.imshow("Homography", homography) 
cv2.imshow("Img", img1) 
cv2.waitKey(1000)
cv2.destroyAllWindows()


In [9]:
matrix


array([[ 1.00446597e+00,  1.19792770e-03, -3.42232934e+00],
       [ 4.63527446e-05,  1.00319255e+00, -7.69090901e-01],
       [ 4.14495514e-06,  1.07029465e-06,  1.00000000e+00]])

In [10]:
img3 = cv2.drawMatches(img1, kp_img1, img2, kp_img2, good_points, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("lines", img3)
cv2.waitKey(1000)
cv2.imwrite("homography.png", img3)
cv2.destroyAllWindows()
